<a href="https://www.kaggle.com/code/sivm205/e-commerce-sales-analysis?scriptVersionId=174726222" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
df = pd.read_csv("/kaggle/input/e-commerce-website-sales-data/ECOMM DATA.xlsx - Orders.csv")
df.head()

In [ ]:
df.columns

#### Description of the dataset

 -ROW ID: A unique identifier for each row in the dataset.
- Order ID: A unique identifier for each order.
- Order Date: The date on which the order was placed.
- Ship Date: The date on which the order was shipped.
- Ship Mode: The shipping mode used for the order.
- Customer ID: A unique identifier for each customer.
- Customer Name: The name of the customer who placed the order.
- Segment: The market segment to which the customer belongs.
- City: The city in which the customer is located.
- State: The state in which the customer is located.
- Country: The country in which the customer is located.
- Market: The market in which the order was placed.
- Postal Code: The postal code of the customer's location.
- Region: The region in which the customer is located.
- Product ID: A unique identifier for each product.
- Category: The category to which the product belongs.
- Sub-Category: The sub-category to which the product belongs.
- Product Name: The name of the product.
- Sales: The total sales amount for the order.
- Quantity: The quantity of the product ordered.
- Discount: The discount applied to the order.
- Shipping Cost: The cost of shipping the order.
- Profit: The profit earned from the order.
- Order Priority: The priority of the order.

This dataset contains information about orders placed by customers, including details about the customers, products, and orders themselves. It includes information such as the order date, shipping details, customer information, product details, and sales information. The dataset can be used to analyze sales trends, customer behavior, and other aspects of the business.

### Before Diving into the Analysis, Address the Following Prerequisites:
Objective of the Analysis: Understand why this analysis is crucial from a business perspective. For instance, you might want to answer questions such as:
Which product is performing the best and which products are doing well?
Which region has the highest product sales?
Which category yields the highest profit margin in a particular region, and so on.
These questions will help define the direction of your analysis and ensure it aligns with your business goals. Remember, a well-defined objective is the first step towards a successful analysis.

Once you get the idea about the objective of the dataset, now its time to validate that the dataset must contain all the important, relavant attributes to answers all these questions.since we have all kind of attributes we directly jump into the analysis part

In [ ]:
df.info()

In [ ]:
#removing postal Code as it contain high null value
df['Postal Code'].isnull().sum()

In [ ]:
df = df.drop('Postal Code', axis= 1)
df.info()

In [ ]:
def return_type():
    ls = {}
    for i in df.columns:
        ls[i] = type(df[i][0])
    return ls
return_type()

In [ ]:
#there are some wrong data type are present which needed to change before the analysis such as data, order id and so on
def return_type():
    ls = {}
    for i in df.columns:
        ls[i] = df[i].dtype
    return ls
return_type() 

#instead  use this 'df.dtypes' for single line statement, u will get the same output

In [ ]:
df.describe(include='all')

In [ ]:
#order date, ship date, ship mode,Customer name, Segment, city, state, category, sub category, order priority. These data types needs to be changed for accurate analysis

df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df['Ship Mode'] = df['Ship Mode'].astype("category")
df['Customer Name'] = df['Customer Name'].astype('str')
df['Segment'] = df['Segment'].astype("category")
df['City'] = df['City'].astype("category")
df['State'] = df['State'].astype("category")
df['Category'] = df['Category'].astype("category")
df['Sub-Category'] = df['Sub-Category'].astype("category")
df['Order Priority'] = df['Order Priority'].astype("category")


In [ ]:
df.dtypes

In [ ]:
#find out duplicate rows in the dataset
duplicates = df[df.duplicated()]
duplicates #no duplicate rows are present in the data

In [ ]:

duplicate_rows = df[df.duplicated()]

if not duplicate_rows.empty:
    # Remove duplicate rows
    df.drop_duplicates(inplace= False)
    print(f'Removed {len(duplicate_rows)} duplicate rows')
else:
    print('No duplicate rows found')

In [ ]:
#find the missing values 
df.isnull().sum()
#no missing value are present in the dataset

In [ ]:
#set row id as index for the dataset
if df['ROW ID'].is_unique:
    df.set_index(['ROW ID'], inplace= True)
    print("set index successfully\n")
else:
    df.drop_duplicates('ROW ID', inplace= True)
    print('removed duplicates from the row id')
    df.set_index(  df['ROW ID'], inplace= True)
    

In [ ]:
df.head(5) #it still containing row id as a column

In [ ]:
new_df = df.drop('ROW ID', axis= 1)
new_df.head(5)

In [ ]:
duplicates_df = new_df[new_df.duplicated()]
len(duplicates_df)

In [ ]:
new_df.head(5)

In [ ]:
#proceeding with the numerical attributes
# first find out outliers 
#there are three ways to detect outliers in the dataframe, I am gonna use interquartile range method
# outliers = 1.5 x IQR Rule = [IQR = Q3 - Q1 ]
# anything below Q1 - (IQR x 1.5)  &&
#anything above Q3 + (IQR x1.5) are treadted as outliers and direct validate through box plots

In [ ]:
numerical_features = new_df.select_dtypes(include='number')
numerical_features

In [ ]:
#visualise the outliers using box plot 
plt.figure(figsize=(10, 6))
numerical_features.boxplot()
plt.title('Boxplot of E-Commerce Order Data')
plt.xlabel('Feature')
plt.ylabel('Value')
plt.xticks(rotation=45)
plt.show()

### ------------------------------------------ estimate outliers for each numerical columns-------------------------------------

In [ ]:
#visualise individually 
import plotly.express as px
def visualise_outliers(df, attribute):
    fig = px.box(df[attribute])
    fig.show()
    
    
for col in numerical_features:
    visualise_outliers(new_df,col)

In [ ]:
#start with the sales

sns.histplot(new_df['Sales'])
plt.show()

In [ ]:
#since sales data are highly skewed because of the high range of outliers. to remove them I am gonna use IQR methodbb
IQR = np.percentile(new_df['Sales'], 75) - np.percentile(new_df['Sales'], 25)
upper_bound_s = np.percentile(new_df['Sales'], 75) + ( IQR*1.5)
lower_bound_s = np.percentile(new_df['Sales'], 25) - (IQR*15)

In [ ]:
IQR, lower_bound_s , upper_bound_s

In [ ]:
Sale = new_df #creating a new instance of df
Sale[ (Sale['Sales']<lower_bound_s) & (Sale['Sales']>upper_bound_s)] = np.nan 

In [ ]:
Sale.dropna(inplace = True)

In [ ]:
#now visualise the sales data
sns.histplot(Sale["Sales"], kde=True)


In [ ]:
Sale['Sales']

In [ ]:
numerical_features[(new_numerical_features['Sales']<upper_bound_s) & (new_numerical_features['Sales']>lower_bound_s)]

In [ ]:
new_df1 = new_df[(new_df['Sales']<upper_bound_s) & (new_df['Sales']>lower_bound_s)]
new_df1

In [ ]:
from plotly.express import *
box(new_df1['Sales'])


In [ ]:
#calculate IQR for numerical Features
IQR = np.percentile(numerical_features, 75) - np.percentile(numerical_features, 25)
IQR

In [ ]:
upper_bound = np.percentile(numerical_features, 75) + ( IQR*1.5)
lower_bound = np.percentile(numerical_features, 25) - (IQR*15)

In [ ]:
lower_bound ,upper_bound

In [ ]:
new_numerical_features = numerical_features[(numerical_features>lower_bound) & (numerical_features<upper_bound)]
new_numerical_features

In [ ]:
#removing the outliers
removed_outliers_df = new_numerical_features.dropna()

In [ ]:
removed_outliers_df

In [ ]:
sns.histplot(new_df['Sales'], kde=True) #with outliers
plt.title('Distribution of Sales before reomving of outliers')
plt.xlabel('Sales')
plt.show()

In [ ]:
#without outliers
sns.histplot(removed_outliers_df['Sales'], kde=True)
plt.title('Distribution of Sales without outliers')
plt.xlabel('Sales')
plt.show()

### from the above visualisation it is clear that all the outliers are now removed

In [ ]:
removed_outliers_df.describe()

In [ ]:
removed_outliers_df.isnull().sum()

In [ ]:
#lets see once again all the outliers in the numerical columns
sns.boxplot(removed_outliers_df)